<a href="https://colab.research.google.com/github/scancer-org/data-eng-preparation/blob/main/PCAM_Dataset_preparation_(HDF5_to_PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### TODO:
# Load data - this is the major phase here
# Sample out of the data - like 100 examples for train, 20 example for test, same distribution of classes
# Have some stats on it
# Create (in PyTorch) a base model
# Train
# Eval
# Test

In [2]:
# Test labels (from original, which I'll split for train / test)
import h5py
import numpy as np
import torch
from google.colab import drive
from torch.utils import data
from os import listdir
from pathlib import Path
from torch.utils.data import DataLoader, Dataset

# anyeone interested should have this path configured already, dataset can be downloaded from GitHub / Drive (PCAM)
# Batch Size = 128
drive_base_path = 'gdrive/MyDrive/pcamv1/'
BATCH_SIZE = 128

In [3]:
drive.mount('/content/gdrive/')
!ls gdrive/MyDrive/pcamv1

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
camelyonpatch_level_2_split_test_meta.csv
camelyonpatch_level_2_split_test_x.h5
camelyonpatch_level_2_split_test_y.h5
camelyonpatch_level_2_split_train_mask.h5
camelyonpatch_level_2_split_train_meta.csv
camelyonpatch_level_2_split_train_x.h5
camelyonpatch_level_2_split_train_y.h5
camelyonpatch_level_2_split_valid_meta.csv
camelyonpatch_level_2_split_valid_x.h5
camelyonpatch_level_2_split_valid_y.h5


In [4]:
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_valid_x.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_test_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_valid_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_test_x.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_mask.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_x.h5.gz

In [5]:
class H5Dataset_From_2_Files_X_Y(Dataset):
    def __init__(self, path):
        self.file_path = path
        self.dataset_x = None
        self.dataset_y = None
        ### Going to read the X part of the dataset - it's a different file
        with h5py.File(self.file_path + '_x.h5', 'r') as filex:
            self.dataset_x_len = len(filex['x'])

        ### Going to read the y part of the dataset - it's a different file
        with h5py.File(self.file_path + '_y.h5', 'r') as filey:
            self.dataset_y_len = len(filey['y'])

    def __getitem__(self, index):
        if self.dataset_x is None:
            self.dataset_x = h5py.File(self.file_path +'_x.h5', 'r')['x']
        if self.dataset_y is None:
            self.dataset_y = h5py.File(self.file_path +'_y.h5', 'r')['y']
        return (self.dataset_x[index], self.dataset_y[index])

    def __len__(self):
        assert self.dataset_x_len == self.dataset_y_len
        return self.dataset_x_len

In [6]:
dataloader_params = {'batch_size': BATCH_SIZE, 'shuffle': True, 'num_workers': 2}

train_path = drive_base_path + 'camelyonpatch_level_2_split_train'
val_path = drive_base_path + 'camelyonpatch_level_2_split_valid'
test_path = drive_base_path + 'camelyonpatch_level_2_split_test'

test_dataset = H5Dataset_From_2_Files_X_Y(test_path)
test_dataloader = DataLoader(test_dataset, **dataloader_params)

val_dataset = H5Dataset_From_2_Files_X_Y(val_path)
val_dataloader = DataLoader(val_dataset, **dataloader_params)

train_dataset = H5Dataset_From_2_Files_X_Y(train_path)
train_dataloader = DataLoader(train_dataset, **dataloader_params)

In [7]:
for x, y in test_dataloader:
  print(x.shape)
  print(y.shape)
  break

for x, y in val_dataloader:
  print(x.shape)
  print(y.shape)
  break

for x, y in train_dataloader:
  print(x.shape)
  print(y.shape)
  break

torch.Size([128, 96, 96, 3])
torch.Size([128, 1, 1, 1])
torch.Size([128, 96, 96, 3])
torch.Size([128, 1, 1, 1])
torch.Size([128, 96, 96, 3])
torch.Size([128, 1, 1, 1])
